<a href="https://colab.research.google.com/github/anuroop-ghosh/WBS-MSBA-AIP-Assignment-Brazilian-eCommerce/blob/main/AIP_Group_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geohash2
!pip install geopy

  Preparing metadata (setup.py) ... done
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15543 sha256=5224776453339c4770074b06f3434fdd598580a7a5d43f6106f0406177058d09
  Stored in directory: /root/.cache/pip/wheels/c0/21/8d/fe65503f4f439aef35193e5ec10a14adc945e20ff87eb35895
Successfully built geohash2


## Adding all libraries
 - This is all libraries needed for the project. Last line allows uploading files directly from devices without going to files on colab

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import geohash2 as geohash
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt
from geopy.distance import geodesic
from sklearn.svm import SVC
svm_algo = SVC(C=1.0, kernel='rbf', gamma=1e-3)
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from xgboost import XGBClassifier as XGB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection  import train_test_split
from sklearn.tree import DecisionTreeClassifier as tree
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import files
upload = files.upload()

## Reading and printing all csv files individually
- if the files.upload above isn't working, this will make sure that the files are being uploaded to the project


In [ ]:
customers = pd.read_csv("olist_customers_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
translation = pd.read_csv("product_category_name_translation.csv")

In [ ]:
print("\n Customers Dataset \n\n", customers.isnull().sum())
#print("\n Geolocation Dataset \n\n", geolocation_dataset.isnull().sum())
print("\n Order Items Dataset \n\n", order_items.isnull().sum())
print("\n Order Payments Dataset \n\n", order_payments.isnull().sum())
print("\n Order Reviews Dataset \n\n", order_reviews.isnull().sum())
print("\n Orders Dataset \n\n", orders.isnull().sum())
print("\n Products Dataset \n\n", products.isnull().sum())
print("\n Sellers Dataset \n\n", sellers.isnull().sum())


 Customers Dataset 

 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

 Order Items Dataset 

 order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

 Order Payments Dataset 

 order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

 Order Reviews Dataset 

 review_id                      0
order_id                       0
review_score                   0
review_comment_title       79404
review_comment_message     52429
review_creation_date           0
review_answer_timestamp        0
dtype: int64

 Orders Dataset 

 order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("\n Customers Dataset \n\n", customers.info())
print("\n Order Items Dataset \n\n", order_items.info())
print("\n Order Payments Dataset \n\n", order_payments.info())
print("\n Order Reviews Dataset \n\n", order_reviews.info())
print("\n Orders Dataset \n\n", orders.info())
print("\n Products Dataset \n\n", products.info())
print("\n Sellers Dataset \n\n", sellers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB

 Customers Dataset 

 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_

The above has completed the data import and overview

---



##Calculate and label the volumetric weight of the products

Calculating the volumetric weight of each product and assigning a rating to it.

$\text{Volumetric Weight} = \frac{\text{Length} \times \text{Width} \times \text{Height}}{5}$

Calculating this we get the volumetric weight in grams. After this we assign weights to each product's volumetric weight based upon this:-

$
\begin{array}{|c|c|}
\hline
Range & Weight \\
\hline
Below 500g &  1 \\
\hline
500g - 1500g &  2 \\
\hline
1500g - 3000g &  3 \\
\hline
3000g - 5000g &  4 \\
\hline
Above 5000g &  5 \\
\hline
\end{array}
$

In [ ]:
#Calculating the volumetric weight
products['product_volumetric_weight_g'] = (products['product_length_cm'] * products['product_width_cm'] * products['product_height_cm'])/5

#Ranges are Below 500g, 500-1500g, 1500-3000g, 3000-5000g and above 5000g
def volumetric_weight_rating(volumetric_weight):
    if volumetric_weight < 500:
      return 1
    elif volumetric_weight < 1500:
      return 2
    elif volumetric_weight < 3000:
      return 3
    elif volumetric_weight < 5000:
      return 4
    else:
      return 5

#Applying the volumetric weight rating
products['volumetric_weight_rating'] = products['product_volumetric_weight_g'].apply(volumetric_weight_rating)
print(products)

                             product_id              product_category_name  \
0      1e9e8ef04dbcff4541ed26657ea517e5                         perfumaria   
1      3aa071139cb16b67ca9e5dea641aaa2f                              artes   
2      96bd76ec8810374ed1b65e291975717f                      esporte_lazer   
3      cef67bcfe19066a932b7673e239eb23d                              bebes   
4      9dc1a7de274444849c219cff195d0b71              utilidades_domesticas   
...                                 ...                                ...   
32946  a0b7d5a992ccda646f2d34e418fff5a0                   moveis_decoracao   
32947  bf4538d88321d0fd4412a93c974510e6  construcao_ferramentas_iluminacao   
32948  9a7c6041fa9592d9d9ef6cfe62a71f8c                    cama_mesa_banho   
32949  83808703fc0706a22e264b9d75f04a2e             informatica_acessorios   
32950  106392145fca363410d287a815be6de4                    cama_mesa_banho   

       product_name_lenght  product_description_lenght  product

In [ ]:
# Calculate product volume
products['product_volume_cm3'] = products['product_length_cm'] * products['product_width_cm'] * products['product_height_cm']

# Define a function to rate
print(products[['product_weight_g','product_volume_cm3','product_description_lenght']].describe())

def weight_rating(weight):
    if weight < 300:
        return 1
    elif weight < 700:
        return 2
    elif weight < 1900:
        return 3
    else:
        return 4

def volume_rating(volume):
    if volume < 2880:
        return 1
    elif volume < 6840:
        return 2
    elif volume < 18480:
        return 3
    else:
        return 4

def descleng_rating(des_lenghth):
    if des_lenghth < 339:
        return 1
    elif des_lenghth < 595:
        return 2
    elif des_lenghth < 972:
        return 3
    else:
        return 4

# Application rating function
products['weight_rating'] = products['product_weight_g'].apply(weight_rating)
products['volume_rating'] = products['product_volume_cm3'].apply(volume_rating)
products['descleng_rating'] = products['product_description_lenght'].apply(volume_rating)

# Inspection: Output processed data
volume_rating_counts = products['volume_rating'].value_counts().sort_index()
weight_rating_counts = products['weight_rating'].value_counts().sort_index()
descleng_rating_counts = products['descleng_rating'].value_counts().sort_index()
print(volume_rating_counts,weight_rating_counts,descleng_rating_counts)


       product_weight_g  product_volume_cm3  product_description_lenght
count      32949.000000        32949.000000                32341.000000
mean        2276.472488        16564.096695                  771.495285
std         4282.038731        27057.041650                  635.115225
min            0.000000          168.000000                    4.000000
25%          300.000000         2880.000000                  339.000000
50%          700.000000         6840.000000                  595.000000
75%         1900.000000        18480.000000                  972.000000
max        40425.000000       296208.000000                 3992.000000
volume_rating
1    8205
2    8265
3    8232
4    8249
Name: count, dtype: int64 weight_rating
1    7211
2    8929
3    8550
4    8261
Name: count, dtype: int64 descleng_rating
1    31782
2      559
4      610
Name: count, dtype: int64


## Translation
- Using dictionary to directly translate the product names to english without doing that on excel

In [ ]:
#Translate to english
# Create the translation dictionary
Translation_dict = dict(zip(translation['product_category_name'], translation['product_category_name_english']))

# Map the 'product_category_name' to its English equivalent
translation['product_category_english'] = translation['product_category_name'].map(Translation_dict)

# Handle missing translations by falling back to the original 'product_category_name'
translation['product_category_english'] = translation['product_category_name_english'].fillna(translation['product_category_name'])

# Check the output
print(translation[['product_category_name', 'product_category_english']].head())

    product_category_name product_category_english
0            beleza_saude            health_beauty
1  informatica_acessorios    computers_accessories
2              automotivo                     auto
3         cama_mesa_banho           bed_bath_table
4        moveis_decoracao          furniture_decor


##Calculate the distance between zipcodes

Calculating the distance between the zipcodes of the customer with the zipcode of the seller using the latitude and longitude data provided in the geolocation dataset.

In [ ]:
# Merge the orders and customers datasets on the customer ID to get customer zip code prefixes in the orders DataFrame.
orders = pd.merge(orders, customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id', how='left')

# Merge the orders dataset with sellers on the 'order_id' column
# This will bring the 'seller_zip_code_prefix' into the orders dataset
orders = pd.merge(orders, order_items[['order_id', 'seller_id']], on='order_id', how='left')
orders = pd.merge(orders, sellers[['seller_id', 'seller_zip_code_prefix']], on='seller_id', how='left')

# Create a dictionary to efficiently map zip codes to coordinates
# Group by 'geolocation_zip_code_prefix' and aggregate coordinates (e.g., using mean)
geolocation_dict = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().to_dict('index')

def calculate_distance(row):
    customer_zip = row['customer_zip_code_prefix']
    seller_zip = row['seller_zip_code_prefix']

# Handle missing zip codes:
    if pd.isna(customer_zip) or pd.isna(seller_zip):
        return np.nan  # Return NaN if either zip code is missing

    # Access the latitude and longitude values directly
    try:  # Try to access coordinates, if available
        customer_coords = (geolocation_dict[customer_zip]['geolocation_lat'], geolocation_dict[customer_zip]['geolocation_lng'])
        seller_coords = (geolocation_dict[seller_zip]['geolocation_lat'], geolocation_dict[seller_zip]['geolocation_lng'])
        return geodesic(customer_coords, seller_coords).km
    except KeyError:  # Handle KeyError if zip code not in dictionary
        return np.nan  # Return NaN if zip code not found

    return geodesic(customer_coords, seller_coords).km

# Calculate distances for each order
orders['distance_km'] = orders.apply(calculate_distance, axis=1)

##Calculate the duration of each process

Due to logical errors found in the timestamp of the original data during the initial inspection, such as customers receiving packages earlier than merchants, this resulted in over 3000 instances of such data. They are returned as 0

In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

order_reviews['review_creation_date'] = pd.to_datetime(order_reviews['review_creation_date'])
order_reviews['review_answer_timestamp'] = pd.to_datetime(order_reviews['review_answer_timestamp'])

# Calculate the duration (in hours)
orders['approval_duration_hours'] = (orders['order_approved_at'] - orders['order_purchase_timestamp']).dt.total_seconds() / 3600

# Order shipping time from merchant
orders['delivery_fromseller'] = (
    np.where(
        orders['order_delivered_carrier_date'] >= orders['order_approved_at'],
        (orders['order_delivered_carrier_date'] - orders['order_approved_at']).dt.total_seconds() / 3600,
        0
    )
)

# Delivery time of express delivery
orders['delivery_tocusto'] = (
    np.where(
        orders['order_delivered_customer_date'] >= orders['order_delivered_carrier_date'],
        (orders['order_delivered_customer_date'] - orders['order_delivered_carrier_date']).dt.total_seconds() / 3600,
        0
    )
)

#Calculate the duration of the review answer
order_reviews['review_toanswer'] = (order_reviews['review_answer_timestamp'] - order_reviews['review_creation_date']).dt.total_seconds() / 3600

# Compare order_delivered_custom_date and order_ estimated d_delivery_date
orders['estimated_delivery_later'] = (orders['order_estimated_delivery_date'] > orders['order_delivered_customer_date']).astype(int) #Mark 1 if arrive in time


##Label textual variables

In [ ]:
#Mark the content of the review
order_reviews['title_flag'] = order_reviews['review_comment_title'].notna().astype(int)
order_reviews['message_flag'] = order_reviews['review_comment_message'].notna().astype(int)

#Mark the order status
status_mapping = {
    'created': 1,
    'unavailable': 2,
    'canceled': 3,
    'approved': 4,
    'invoiced': 5,
    'processing': 6,
    'shipped': 7,
    'delivered': 8
}

#Convert text to numbers
orders['order_status_numeric'] = orders['order_status'].map(status_mapping)



payment_mapping = {
    'not_defined': 0,
    'voucher': 1,
    'boleto': 2,
    'credit_card': 3,
    'debit_card': 4
}
#Convert text to numbers
order_payments['payment_type_numeric'] = order_payments['payment_type'].map(payment_mapping)


##Calculate customers' loyalty
- As a group, we thought that this will be a nice addtion to differentiate customers based on loyalty using the link between unique id & customer id in order to merge this into the dataset



In [ ]:
# Merge customers with the loyalty information
customers['customer_loyalty'] = customers.groupby('customer_unique_id')['customer_id'].transform('count')

##Separate orders with one/mulitple review scores
- The code below seperates the orders with one review from those with multiple reviews using order id and checking duplictes and repeated orders as well.
- Right after that the results were converted to two dataframes and the average was calculated by using order id and review score.
- The final step was to merge all datasets as discussed in order to join both merged datasets into one final dataset that will be used to find all results

In [ ]:
# Create a dictionary with order_id as the key and corresponding rows as the value
order_reviews_dict = {}

# Traverse the data box and store each row by order_id
for index, row in order_reviews.iterrows():
    order_id = row['order_id']
    if order_id not in order_reviews_dict:
        order_reviews_dict[order_id] = []
    order_reviews_dict[order_id].append(row)

# Create two lists to store classification results
repeated_orders = []
unique_orders = []

# Traverse the dictionary to determine if order_id is duplicated
for order_id, rows in order_reviews_dict.items():
    if len(rows) > 1:
        repeated_orders.extend(rows)
    else:
        unique_orders.extend(rows)

repeated_orders_df = pd.DataFrame(repeated_orders)
unique_orders_df = pd.DataFrame(unique_orders)

average_repeated_orders = repeated_orders_df.groupby('order_id')['review_score'].mean().reset_index()

# Add English translations to the 'products' DataFrame
products = products.merge(
    translation[['product_category_name', 'product_category_name_english']],
    on='product_category_name',
    how='left'
)

# Generate the final datasets with translations included
repeated_orders_df.drop('review_score', axis=1, inplace=True)
repeated_orders_df = repeated_orders_df.merge(average_repeated_orders, on='order_id')

repeat_df = (
    repeated_orders_df
    .merge(orders, on='order_id', how='inner')  # Use modified 'orders' with calculated durations
    .merge(order_payments, on='order_id', how='inner')
    .merge(customers, on='customer_id', how='inner')
    .merge(order_items, on='order_id', how='inner')
    .merge(products, on='product_id', how='inner')  # 'products' now includes English translations
)

unique_df = (
    unique_orders_df
    .merge(orders, on='order_id', how='inner')  # Use modified 'orders' with calculated durations
    .merge(order_payments, on='order_id', how='inner')
    .merge(customers, on='customer_id', how='inner')
    .merge(order_items, on='order_id', how='inner')
    .merge(products, on='product_id', how='inner')  # 'products' now includes English translations
)

# Check the output
print(repeat_df[['product_category_name', 'product_category_name_english']].head())
print(unique_df[['product_category_name', 'product_category_name_english']].head())


  product_category_name product_category_name_english
0       cama_mesa_banho                bed_bath_table
1       cama_mesa_banho                bed_bath_table
2         esporte_lazer                sports_leisure
3         esporte_lazer                sports_leisure
4      moveis_decoracao               furniture_decor
  product_category_name product_category_name_english
0            cool_stuff                    cool_stuff
1              pet_shop                      pet_shop
2      moveis_decoracao               furniture_decor
3            perfumaria                     perfumery
4    ferramentas_jardim                  garden_tools


##Joining datasets
- Below we can see that we dropped/removed the data we believe that will not be needed and finally merged both sets (unique df , repeat df) into one final set which is called final df




In [ ]:
selected_columns = [
    'review_score',
    'title_flag',
    'message_flag',
    'review_toanswer',
    'order_status_numeric',
    'customer_loyalty',

    'distance_km',
    'approval_duration_hours',
    'delivery_fromseller',
    'delivery_tocusto',
    'estimated_delivery_later',

    'payment_sequential',
    'payment_type',
    'payment_installments',
    'payment_value',
    'payment_type_numeric',

    #'customer_city',
    'customer_state',

    'order_item_id',
    'price',
    'freight_value',
    'product_name_lenght',
    'product_description_lenght',
    'product_photos_qty',
    'product_volumetric_weight_g',
    'volumetric_weight_rating',
    'weight_rating',
    'volume_rating',
    'descleng_rating'
]

#Select the desired column from repetiti_final.df
final_df1 = repeat_df[selected_columns]
final_df2 = unique_df[selected_columns]

#Total final dataset
final_df = pd.concat([final_df1,final_df2], ignore_index=True)

In [ ]:
#Divide the final test dataset based on order status
nodelivery_df = final_df[final_df['order_status_numeric'].isin([1, 2, 3, 4, 5, 6])]
delivery_df = final_df[final_df['order_status_numeric'].isin([7,8])]

#Lasso


*   The lasso model, as the final step in feature engineering, aims to initially identify variables that significantly impact review scores.

☝However, the results show an MSE of 1.19, which is relatively high given that the target variable ranges from 1 to 5, making it less convincing and only useful as a reference. Therefore, we will continue to explore other models.



##Orders without delivery time

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso # Import Lasso class from sklearn.linear_model
from sklearn.model_selection import train_test_split # Import train_test_split
from sklearn.metrics import mean_squared_error # Import mean_squared_error


X1 = nodelivery_df.drop(['review_score'], axis=1)  # All features
y1 = nodelivery_df['review_score']  # Target variable

# Handling missing values
X1.fillna(0, inplace=True)
X1 = pd.get_dummies(X1, drop_first=True)  # Exclusive hot encoding

# Standardized features
scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_scaled, y1, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=5)  #cross validation to find best alpha
lasso_cv.fit(X1_train, y1_train)

LassoCV(cv=5)

In [ ]:
lasso1 = Lasso(alpha= lasso_cv.alpha_) #using the best alpha value
lasso1.fit(X1_train, y1_train)

y1_pred = lasso1.predict(X1_test)
mse1 = mean_squared_error(y1_test, y1_pred)
print(f'Mean Squared Error: {mse1}')

coefficients = pd.DataFrame(lasso1.coef_, index=X1.columns, columns=['Coefficient'])
print(coefficients[coefficients['Coefficient'] != 0])

Mean Squared Error: 1.1998478353654742
                          Coefficient
title_flag                   0.024551
message_flag                -0.256881
review_toanswer              0.015275
order_status_numeric        -0.042633
customer_loyalty             0.100921
approval_duration_hours     -0.020567
delivery_fromseller         -0.007798
estimated_delivery_later     0.128246
payment_value               -0.008768
price                       -0.060008
product_name_lenght          0.061239
product_photos_qty          -0.001958
volume_rating                0.003250
descleng_rating             -0.029643
payment_type_credit_card    -0.061685
payment_type_debit_card     -0.016271
payment_type_voucher        -0.016744
customer_state_BA            0.388010
customer_state_CE            0.065002
customer_state_DF           -0.065166
customer_state_ES            0.134185
customer_state_MG            0.136224
customer_state_MS            0.012148
customer_state_PB            0.125840
customer_st

##Orders with delivery time

In [ ]:
X2 = delivery_df.drop(['review_score'], axis=1)  # All features
y2 = delivery_df['review_score']  # Target variable

# Handling missing values
X2.fillna(0, inplace=True)
X2 = pd.get_dummies(X2, drop_first=True)  # Exclusive hot encoding

# Standardized features
scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=5)  #cross validation
lasso_cv.fit(X2_train, y2_train)
print(f'Best alpha: {lasso_cv.alpha_}')

Best alpha: 0.0005942794888468965


In [ ]:
lasso2 = Lasso(alpha= lasso_cv.alpha_) #using the best alpha value
lasso2.fit(X2_train, y2_train)


y2_pred = lasso2.predict(X2_test)
mse2 = mean_squared_error(y2_test, y2_pred)
print(f'Mean Squared Error: {mse2}')


coefficients = pd.DataFrame(lasso2.coef_, index=X2.columns, columns=['Coefficient'])
print(coefficients[coefficients['Coefficient'] != 0])

Mean Squared Error: 1.6613101945513449
                             Coefficient
title_flag                      0.036643
message_flag                   -0.458909
review_toanswer                 0.013025
order_status_numeric            0.103598
customer_loyalty                0.025367
distance_km                     0.012514
approval_duration_hours        -0.012069
delivery_fromseller            -0.104191
delivery_tocusto               -0.171840
estimated_delivery_later        0.271672
payment_sequential             -0.004480
payment_installments           -0.046304
payment_value                  -0.201486
payment_type_numeric            0.003643
order_item_id                  -0.217462
price                           0.130001
product_name_lenght            -0.021856
product_description_lenght      0.027999
product_photos_qty              0.011986
product_volumetric_weight_g     0.007486
volumetric_weight_rating        0.012875
weight_rating                   0.039929
volume_rating     

# Models for orders without delivery time

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.ensemble import GradientBoostingRegressor as GBDT
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor as XGB
from sklearn.metrics import precision_recall_fscore_support

RF_algo = RF()
RF_model = RF_algo.fit(X1_train, y1_train)

GBDT_algo = GBDT()
GBDT_model = GBDT_algo.fit(X1_train, y1_train)

XGB_algo = XGB()
XGB_model = XGB_algo.fit(X1_train, y1_train)

In [ ]:
models1 = [RF_model, GBDT_model, XGB_model]
names = ['Random Forest', 'Gradient Boosted Decision Trees', 'XGBDT']

for i in range(len(models1)):
  print(f"Model: {names[i]}")

  # predict based on training data
  predict = models1[i].predict(X1_train)

   # Calculate regression metrics (MSE and R-squared)
  mse = mean_squared_error(y1_train, predict)
  r2 = r2_score(y1_train, predict)
  print(f"Mean Squared Error: {mse}")
  print(f"R-squared: {r2}")
  print("\n")


Model: Random Forest
Mean Squared Error: 0.10808314121037466
R-squared: 0.9294247446468711


Model: Gradient Boosted Decision Trees
Mean Squared Error: 0.4508594223091796
R-squared: 0.7056014610465425


Model: XGBDT
Mean Squared Error: 0.0014612689422327143
R-squared: 0.9990458324250872




In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# we get a load of warnings running the code so will supress them
import warnings
warnings.filterwarnings("ignore")

# create a hyperparameter search function for re-usability
def random_search(algo, hyperparameters, X_train, Y_train):
  # do the search using 5 folds/chunks
  clf = RandomizedSearchCV(algo, hyperparameters, cv=5, random_state=2015,
                          scoring='precision_macro', n_iter=20, refit=True)

  # pass the data to fit/train
  clf.fit(X_train, Y_train)

  return clf.best_params_

In [ ]:
# Random Forest
RF_tuned_parameters = {
    'n_estimators': randint(50, 500), # Draw from a uniform distribution between 50 and 500
    'max_depth': randint(2, 7),  # Draw from a uniform distribution between 2 and 7
    'min_samples_split': randint(2, 7),  # Draw from a uniform distribution between 2 and 7
    'max_features': ['sqrt', 'log2', None]
}

RF_best_params = random_search(RF_algo, RF_tuned_parameters, X1_train, y1_train)

In [ ]:
# GBDT
GBDT_tuned_parameters = {
    'n_estimators': randint(25, 250), # Draw from a uniform distribution between 50 and 500
    'learning_rate': uniform(loc=0.01, scale=4.99),  # Draw from a uniform distribution between 0.01 and 5
    'criterion': ['friedman_mse', 'squared_error'],
    'max_depth': randint(2, 7)  # Draw from a uniform distribution between 2 and 7
}

GBDT_best_params = random_search(GBDT_algo, GBDT_tuned_parameters, X1_train, y1_train)

In [ ]:
# XGBDT
XGB_tuned_parameters = {
    'n_estimators': randint(25, 250), # Draw from a uniform distribution between 50 and 500
    # eta is learning rate
    'eta': uniform(loc=0.01, scale=4.99),  # Draw from a uniform distribution between 0.01 and 5
    # objective is the same as criterion
    'objective': ['binary:logistic', 'binary:hinge'],
    'max_depth': randint(2, 7)  # Draw from a uniform distribution between 2 and 7
}

XGB_best_params = random_search(XGB_algo, XGB_tuned_parameters, X1_train, y1_train)

Re-run after hyperparameter tuning

In [ ]:
# score the models
models2 = [RF_model, GBDT_model, XGB_model] # redo this or it uses the old models

for i in range(len(models2)):
  print(f"Model: {names[i]}")

  # predict based on training data
  predict = models2[i].predict(X1_train)

  mse = mean_squared_error(y1_train, predict)
  r2 = r2_score(y1_train, predict)

  print(f"Mean Squared Error: {mse}")
  print(f"R-squared: {r2}")
  print("\n")

Model: Random Forest
Mean Squared Error: 0.10808314121037466
R-squared: 0.9294247446468711


Model: Gradient Boosted Decision Trees
Mean Squared Error: 0.4508594223091796
R-squared: 0.7056014610465425


Model: XGBDT
Mean Squared Error: 0.0014612689422327143
R-squared: 0.9990458324250872




In [ ]:
for i in range(len(models2)):
  print(f"Model: {names[i]}")

  # predict based on TEST data
  predict = models2[i].predict(X1_test)

  # Calculate regression metrics (MSE and R-squared)
  mse = mean_squared_error(y1_test, predict)
  r2 = r2_score(y1_test, predict)
  print(f"Mean Squared Error: {mse}")
  print(f"R-squared: {r2}")
  print("\n")

Model: Random Forest
Mean Squared Error: 0.6641310344827587
R-squared: 0.5750976036515786


Model: Gradient Boosted Decision Trees
Mean Squared Error: 0.7117783410629526
R-squared: 0.5446134767334021


Model: XGBDT
Mean Squared Error: 0.6679577052244011
R-squared: 0.5726493494912732




# Models for Orders with delivery time

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.ensemble import GradientBoostingRegressor as GBDT
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor as XGB
from sklearn.metrics import precision_recall_fscore_support

RF_algo = RF()
RF_model_2 = RF_algo.fit(X2_train, y2_train)

GBDT_algo = GBDT()
GBDT_model_2 = GBDT_algo.fit(X2_train, y2_train)

XGB_algo = XGB()
XGB_model_2 = XGB_algo.fit(X2_train, y2_train)

In [ ]:
models3 = [RF_model_2, GBDT_model_2, XGB_model_2]
names = ['Random Forest', 'Gradient Boosted Decision Trees', 'XGBDT']

for i in range(len(models3)):
  print(f"Model: {names[i]}")

  # predict based on training data
  predict = models3[i].predict(X2_train)

   # Calculate regression metrics (MSE and R-squared)
  mse = mean_squared_error(y2_train, predict)
  r2 = r2_score(y2_train, predict)
  print(f"Mean Squared Error: {mse}")
  print(f"R-squared: {r2}")
  print("\n")

Model: Random Forest
Mean Squared Error: 0.10922358580914275
R-squared: 0.9514447590412201


Model: Gradient Boosted Decision Trees
Mean Squared Error: 1.4786758541853369
R-squared: 0.3426560585059204


Model: XGBDT
Mean Squared Error: 0.948154457982659
R-squared: 0.578498839491171




In [ ]:
for i in range(len(models3)):
  print(f"Model: {names[i]}")

  # predict based on TEST data
  predict = models3[i].predict(X2_test)

  # Calculate regression metrics (MSE and R-squared)
  mse = mean_squared_error(y2_test, predict)
  r2 = r2_score(y2_test, predict)
  print(f"Mean Squared Error: {mse}")
  print(f"R-squared: {r2}")
  print("\n")

Model: Random Forest
Mean Squared Error: 0.7410197434282212
R-squared: 0.6712224331110994


Model: Gradient Boosted Decision Trees
Mean Squared Error: 1.48054353186569
R-squared: 0.34310859542293437


Model: XGBDT
Mean Squared Error: 1.0753638179664962
R-squared: 0.5228797846117978




In [ ]:
 #To predict on new, unseen data:
new_data = pd.DataFrame({'feature1': [X2_test], 'feature2': [X1_test],})
new_predictions_rf = RF_model_2.predict(X2_test)
new_predictions_xgb = XGB_model.predict(X1_test)
new_predictions_rf = new_predictions_rf.round()
new_predictions_xgb = new_predictions_xgb.round()
print(new_predictions_rf)
print(new_predictions_xgb)

[4. 1. 4. ... 5. 2. 5.]
[1. 1. 1. 2. 1. 2. 2. 3. 1. 1. 1. 1. 5. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.
 1. 1. 1. 5. 1. 1. 1. 1. 5. 1. 2. 2. 1. 5. 1. 1. 5. 1. 1. 5. 2. 2. 1. 1.
 4. 1. 1. 1. 1. 2. 1. 3. 2. 3. 3. 2. 2. 1. 1. 5. 3. 5. 1. 2. 1. 3. 1. 2.
 2. 3. 1. 1. 2. 1. 1. 5. 1. 2. 2. 1. 1. 1. 1. 2. 2. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 2. 1. 5. 1. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 1. 1. 2. 1. 1. 5. 1. 3. 2. 1. 1. 1.
 1. 1. 2. 1. 2. 5. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 3. 1. 1. 1. 1. 1. 1. 1.
 2. 1. 1. 1. 2. 1. 2. 1. 1. 2. 1. 1. 2. 3. 1. 1. 1. 2. 1. 1. 2. 1. 1. 1.
 1. 5. 1. 4. 2. 1. 1. 1. 1. 1. 5. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1.
 1. 1. 1. 3. 2. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 4. 1.
 2. 0. 1. 5. 2. 1. 1. 2. 1. 1. 5. 3. 1. 1. 1. 1. 2. 2. 2. 1. 2. 1. 3. 1.
 3. 1. 2. 3. 2. 1. 2. 1. 5. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 3. 1. 1. 3. 1.
 1. 1. 1. 1. 1. 2. 1. 2. 1. 2. 1. 1. 5. 1. 1. 1. 1. 1. 1. 5. 1. 1. 1. 1.
 2. 1. 1. 1. 1. 1. 1. 2. 3.

In [ ]:
print(new_predictions_rf.round())

[4. 1. 4. ... 5. 2. 5.]


##Feature Importance



*   Regarding the variables of review, such as ***title_flag***, ***message_flag***, and the duration of responding to invitation reviews '***review_toanswer***',

☝If their importance ranks high in the model, it can be concluded that **using the habit of writing reviews to draw user profiles can identify customers who habitually score high, and later send invitation reviews specifically**



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Obtain feature importance
importances = XGB_model.feature_importances_
feature_names = X1.columns

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)


                        Feature  Importance
28            customer_state_BA    0.392621
21     volumetric_weight_rating    0.114544
29            customer_state_CE    0.074598
8              delivery_tocusto    0.060418
4              customer_loyalty    0.055183
33            customer_state_MA    0.024593
17          product_name_lenght    0.017901
16                freight_value    0.015625
19           product_photos_qty    0.015061
14                order_item_id    0.014751
1                  message_flag    0.014721
20  product_volumetric_weight_g    0.014102
31            customer_state_ES    0.013143
32            customer_state_GO    0.012764
46            customer_state_RS    0.011354
39            customer_state_PE    0.011138
15                        price    0.011135
6       approval_duration_hours    0.010668
11         payment_installments    0.010439
3          order_status_numeric    0.009384
12                payment_value    0.009146
47            customer_state_SC 

In [ ]:
importances2 = RF_model_2.feature_importances_

feature_names2 = X2.columns
importance_df2 = pd.DataFrame({'Feature': feature_names2, 'Importance': importances2})

importance_df2 = importance_df2.sort_values(by='Importance', ascending=False)

print(importance_df2)


                        Feature  Importance
1                  message_flag    0.119103
12                payment_value    0.097806
8              delivery_tocusto    0.073236
15                        price    0.070826
9      estimated_delivery_later    0.070732
2               review_toanswer    0.067706
7           delivery_fromseller    0.064491
6       approval_duration_hours    0.054712
5                   distance_km    0.051264
16                freight_value    0.050696
18   product_description_lenght    0.050668
14                order_item_id    0.043889
20  product_volumetric_weight_g    0.042441
17          product_name_lenght    0.035779
11         payment_installments    0.018752
19           product_photos_qty    0.014147
22                weight_rating    0.008901
4              customer_loyalty    0.005506
45            customer_state_RJ    0.005001
0                    title_flag    0.004982
21     volumetric_weight_rating    0.004236
37            customer_state_MG 



---

#Some cross check for report
*   The result shows that '***weight_rating***','***volume_rating***' and
'***volumetric_weight_rating***' hava a significant impact on **order delivery time**

*   Intercept: 1.0343195383056702

*   Coefficients:

 distance_km 0.01030723  

 weight_rating 3.40605733

 volume_rating -4.08864203  

 volumetric_weight_rating 5.16266484

In [ ]:
# Some cross check of variable
import pandas as pd
from sklearn.linear_model import LinearRegression

X = delivery_df[['distance_km', 'weight_rating','volume_rating','volumetric_weight_rating']]
y = delivery_df['delivery_tocusto']

X['distance_km'].fillna(X['distance_km'].mean(), inplace=True)

# Create and fit multiple regression models
model = LinearRegression()
model.fit(X, y)

print('Intercept:', model.intercept_)
print('Coefficients:', model.coef_)

#Intercept: 106.59872879642506
#Coefficients: [ 0.14433439  7.39839911 -2.67161653  2.88305141]


X = delivery_df[['distance_km', 'weight_rating','volume_rating','volumetric_weight_rating']]
y = delivery_df['freight_value']

X['distance_km'].fillna(X['distance_km'].mean(), inplace=True)


model = LinearRegression()
model.fit(X, y)

print('Intercept:', model.intercept_)
print('Coefficients:', model.coef_)

#Intercept: 1.0343195383056702
#Coefficients: [ 0.01030723  3.40605733 -4.08864203  5.16266484]

Intercept: 106.5987287964256
Coefficients: [ 0.14433439  7.39839911 -2.67161653  2.88305141]
Intercept: 1.0343195383055601
Coefficients: [ 0.01030723  3.40605733 -4.08864203  5.16266484]
